### Umformen der UMD31-basierten Genotypdaten in das aktuelle ARS-UCD1.2 Format

Importieren aller vorerst benötigten Anwendungen:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Erstellen von PLINK Dateiensets `(.bed, .bim, .fam)` aus der vorhandenen `.lgen` Datei

In [ ]:
HOME = os.getenv('HOME')
gts_dir = "{}/GTS/".format(HOME)

In [ ]:
gts = pd.read_csv("{0}/namibia_gts_UMD31.csv".format(gts_dir), \
                  sep='\t', low_memory=False)

Erzeugen der `.lgen` und `.fam` Datei:

In [ ]:
gts['allele_1']=gts['gt'].str[0]
gts['allele_2']=gts['gt'].str[1]
gts['fam_id']='0'
gts['sire_id']='0'
gts['dam_id']='0'
gts['sex'] ='0'
gts['pht'] = 99999
gts[['allele_1','allele_2']] = gts[['allele_1','allele_2']].applymap\
(lambda x: '0' if x=='-' else x)
gts[['fam_id','animal_id','snp_id','allele_1','allele_2']].to_csv\
                ("{0}/namibia_UMD31.lgen".format(gts_dir), \
                header=None, sep='\t', index=False)

gts.drop_duplicates(['animal_id'])[['fam_id','animal_id',\
                                    'sire_id','dam_id','sex','pht']].\
                                    to_csv("{0}/namibia_UMD31.fam".format(gts_dir), \
                                    header=None, sep='\t', index=False)

#.map file (for UMD31 assembly) is available as namibia_UMD31.map 

In [ ]:
cmd = "plink --cow --lfile {0}/namibia_UMD31 --out {0}/namibia_UMD31".format(gts_dir)
os.system(cmd)

---

`namibia_UMD31.fam` enthält keine Informationen zu den Geschlechtern. <br>
Mit der PLINK Funktion `--check-sex` und `--impute-sex` können die Informationen in die `.fam` Datei geschrieben werden.

In [ ]:
#Run in bash shell in gts_dir
#plink --cow --bfile namibia_UMD31 --check-sex .5 .7 --out namibia_UMD31
#plink --cow --bfile namibia_UMD31 --impute-sex .5 .7 --make-bed --out namibia_UMD31
#plink --cow --bfile ansbach_UMD31 --check-sex .5 .7 --out ansbach_UMD31
#plink --cow --bfile ansbach_UMD31 --impute-sex .5 .7 --make-bed --out ansbach_UMD31

In [ ]:
namibia_sexcheck = pd.read_csv("{0}/namibia_UMD31.sexcheck".\
                               format(gts_dir), delim_whitespace=True)

In [ ]:
plt.hist(namibia_sexcheck['F'], bins=20)

Lesen der UMD31-basierten `.bim` Dateien für das Ansbach Triesdorfer Rind und Namibia Fleckvieh:

In [ ]:
bim = pd.read_csv("{}/namibia_UMD31.bim".format(gts_dir), sep='\t', header=None)
#bim = pd.read_csv("{}/ansbach_UMD31.bim".format(gts_dir), sep='\t', header=None)
bim.columns = ['bim_chr','marker','bim_cm','bim_bp', 'bim_A1', 'bim_A2']

Einlesen der ARS-UCD1.2-basierten Daten (verfügbar durch: [https://www.animalgenome.org/repository/cattle/UMC_bovine_coordinates/](https://www.animalgenome.org/repository/cattle/UMC_bovine_coordinates/))

In [ ]:
HOME = os.getenv('HOME')
array_dir = "{0}/REF/ARRAYS".format(HOME)
ARS12_MAP = "{0}/9913_ARS1.2_58336_SNP50_marker_name_180910.map".format(array_dir)
ARS12_REF = "{0}/9913_ARS1.2_58336_SNP50_marker_name_180910.REF".format(array_dir)
ARS12_REF_ALLELE = "{0}/9913_ARS1.2_58336_SNP50_marker_name_180910.REF_ALLELE".\
                                format(array_dir)
MAP = pd.read_csv(ARS12_MAP, sep='\t', header=None)
MAP.columns = ['MAP_chr', 'marker','MAP_cm','MAP_bp']
REF = pd.read_csv(ARS12_REF, sep='\t', header=None)
REF.columns=['marker','REF_A','REF_B','REF_A1','REF_A2']
REF_A = pd.read_csv(ARS12_REF_ALLELE,sep='\t',header=None) # Reference allele
REF_A.columns=['marker','REF_A_R']

Verknüpfen der Informationen aus den UMD31-basierten Daten und der ARS-UCD1.2-basierten Daten anhand der Marker:

In [ ]:
ALL = pd.merge(MAP,bim)
ALL = pd.merge(ALL,REF)
ALL = pd.merge(ALL,REF_A)

Prüfen auf andere Allele außer A, C, T und G:

In [ ]:
ALL['REF_A1A2']=ALL.REF_A1 + ALL.REF_A2
ALL.groupby(['REF_A1A2'])['REF_A1A2'].count()

In [ ]:
ALL['bim_A1A2'] = ALL.bim_A1 + ALL.bim_A2
ALL.groupby(['bim_A1A2'])['bim_A1A2'].count()

"0N" stellen InDels dar; AT / TA und CG /GC können rechnerisch nicht unterschieden werden.

---

Überprüfung und Anpassung der Strang-Konformität:

In [ ]:
def c_gt(gt):
    c = {'A':'T','C':'G','G':'C','T':'A'}
    return ''.join([c[x] for x in list(gt)])
new_A1 = []
new_A2 = []

variants_excluded_namibia = []
#variants_excluded_ansbach = []

for i in ALL.index:
    bim_A1A2 = ALL.iloc[i].bim_A1A2
    REF_A1A2 = ALL.iloc[i].REF_A1A2
    MAP_chr = ALL.iloc[i].MAP_chr
    MAP_bp = ALL.iloc[i].MAP_bp
    if bim_A1A2 in ['0A','0C','0G','0T','AT','CG','GC','TA']:
        #variants_excluded_namibia.append("{0}_{1}".format(MAP_chr, MAP_bp))
        variants_excluded_ansbach.append("{0}_{1}".format(MAP_chr, MAP_bp))
        #print(bim_A1A2, MAP_chr, MAP_bp)
        new_A1.append(bim_A1A2[0])
        new_A2.append(bim_A1A2[1]) 
    elif bim_A1A2==REF_A1A2:
        new_A1.append(bim_A1A2[0])
        new_A2.append(bim_A1A2[1])
    elif bim_A1A2==REF_A1A2[::-1]:
        new_A1.append(bim_A1A2[0])
        new_A2.append(bim_A1A2[1])
    else:
        new_A1.append(c_gt(bim_A1A2)[0])
        new_A2.append(c_gt(bim_A1A2)[1])
 
ALL['new_A1'] = new_A1
ALL['new_A2'] = new_A2    

In [ ]:
variants_excluded_namibia = pd.DataFrame({'variant_id':variants_excluded_namibia})
#variants_excluded_ansbach = pd.DataFrame({'variant_id':variants_excluded_ansbach})

Erstellen einer Datei `ARS12_REF_NEW` mit Strang-angepassten Allelen und einer Datei mit ausgeschlossenen Varianten:

In [ ]:
ALL[['marker','bim_A1','bim_A2','new_A1','new_A2']].\
to_csv('{}/ARS12.REF_NEW_namibia'.format(gts_dir), \
       sep='\t', header=None, index=False)
variants_excluded_namibia[['variant_id']].\
to_csv("{0}/variants_excluded_namibia.csv".format(gts_dir), \
       sep='\t', header=None, index=False)
#ALL[['marker','bim_A1','bim_A2','new_A1','new_A2']].\
#to_csv('{}/ARS12.REF_NEW_ansbach'.format(gts_dir), sep='\t', header=None, index=False)
#variants_excluded_namibia[['variant_id']].\
#to_csv("{0}/variants_excluded_ansbach.csv".format(gts_dir), sep='\t', \
#header=None, index=False)

Umwandeln der Markerbezeichung in das Format "CHR_BP" (Chromosom_Basenposition):

In [ ]:
ALL['CHR_BP'] = ALL.MAP_chr.astype(str)+'_'+ALL.MAP_bp.astype(str)
ALL=ALL.drop_duplicates(['CHR_BP'])
ALL[['marker','CHR_BP','REF_A_R']].to_csv('{}/CHR_BP_REF_A'.\
                    format(gts_dir), sep='\t', header=None, index=False)

Aktualisieren aller Datein mit den neuen Informationen mit Hilfe von diversen PLINK --update Befehlen:<br>
+ update-chr [filename] {chr col. number} {variant ID col.} {skip}<br>
+ update-cm [filename] {cm col. number} {variant ID col.} {skip}<br>
+ update-name [filename] {new ID col. number} {old ID col.} {skip}<br>
+ update-map [filename] {bp col. number} {variant ID col.} {skip}<br>
+ update-alleles [filename]<br>

In [ ]:
cmd = "plink --cow --bfile {1}/namibia_UMD31 \
--update-chr {0} 1 2 \
--update-map {0} 4 2 \
--update-cm {0} 3 2 \
--update-alleles {1}/ARS12.REF_NEW_namibia \
--make-bed --out {1}/namibia_ARS12".format(ARS12_MAP, gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {1}/ansbach_UMD31 \
--update-chr {0} 1 2 \
--update-map {0} 4 2 \
--update-cm {0} 3 2 \
--update-alleles {1}/ARS12.REF_NEW_ansbach \
--make-bed --out {1}/ansbach_ARS12".format(ARS12_MAP, gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/namibia_ARS12 \
--update-name {0}/CHR_BP_REF_A 2 1 \
--make-bed --out {0}/namibia_ARS12".format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/ansbach_ARS12 \
--update-name {0}/CHR_BP_REF_A 2 1 \
--make-bed --out {0}/ansbach_ARS12".format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "cat {0}/CHR_BP_REF_A | cut -f2 > {0}/ARS12_variants".format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/namibia_ARS12 \
--extract {0}/ARS12_variants --make-bed --out {0}/namibia_ARS12".format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/ansbach_ARS12 \
--extract {0}/ARS12_variants --make-bed --out {0}/ansbach_ARS12".format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/namibia_ARS12 \
--exclude {0}/variants_excluded_namibia.csv --make-bed --out {0}/namibia_ARS12".\
format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/ansbach_ARS12 \
--exclude {0}/variants_excluded_ansbach.csv --make-bed 
--out {0}/ansbach_ARS12".format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/namibia_ARS12 \
--exclude {0}/namibia_ansbach_ARS12-merge.missnp --make-bed 
--out {0}/namibia_ARS12".format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/ansbach_ARS12 \
--exclude {0}/namibia_ansbach_ARS12-merge.missnp --make-bed \
--out {0}/ansbach_ARS12".format(gts_dir)
#os.system(cmd)

In [ ]:
cmd = "plink --cow --bfile {0}/namibia_ARS12 --bmerge {0}/ansbach_ARS12 \
--make-bed --out {0}/namibia_ansbach_ARS12".format(gts_dir)
#os.system(cmd)
cmd = "plink --cow --bfile {0}/namibia_ansbach_ARS12 \
--bmerge {0}/fleckvieh_braunvieh_ARS12 --make-bed \
--out {0}/four_pops".format(gts_dir)
#os.system(cmd)

Ein PLINK Dateienset `(five_pops)` ist nun für folgende fünf Populationen verfügbar:<br>
+ Original Fleckvieh (früheres Fleckvieh)
+ heutiges Fleckvieh
+ Namibia Fleckvieh
+ Brown Swiss
+Holstein Friesian

----

### Hauptkomponentenanalyse

Verwendung des Moduls `PCA` aus der Anwendung `sklearn` zur Durchführung für drei simulierte Populationen (pop1, pop2, pop3):

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

In [ ]:
gts = np.zeros((1500,1000))

In [ ]:
allele_freqs = np.arange(0.01,1.0,0.05)

In [ ]:
np.random.seed(1)
i = 0
for i in range(0,1000):
    pop1 = np.random.choice(3, 500, p=get_gt_freqs(np.random.choice(allele_freqs)))
    pop2 = np.random.choice(3, 500, p=get_gt_freqs(np.random.choice(allele_freqs)))
    pop3 = np.random.choice(3, 500, p=get_gt_freqs(np.random.choice(allele_freqs)))
    gts[0:500, i] = pop1
    gts[500:1000, i] = pop2
    gts[1000:1500, i] = pop3

In [ ]:
pca.fit(gts)

In [ ]:
Y = pca.transform(gts)

In [ ]:
plt.scatter(Y[:500,0],Y[:500,1], color='blue', label='pop1')
plt.scatter(Y[500:1000,0],Y[500:1000,1], color='green', label='pop2')
plt.scatter(Y[1000:,0],Y[1000:,1], color='orange', label='pop3')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()

Anwendung der Hauptkomponentenanalyse auf die zu untersuchenden Populationen:

In [ ]:
cmd = "plink --cow --bfile {0}/five_pops --geno 0.1 --pca \
--out {0}/five_pops".format(gts_dir)
os.system(cmd)

In [ ]:
eigenvecs = pd.read_csv("{0}/five_pops.eigenvec".format(gts_dir), \
                        delim_whitespace=True,header=None)

In [ ]:
out=plt.scatter(eigenvecs[2], eigenvecs[3])

In [ ]:
pop_info = pd.read_excel("{0}/fams/pop_info.xlsx".format(gts_dir))

In [ ]:
pop_info[pop_info['valid']==1].groupby(['pop']).count()

In [ ]:
eigenvecs.rename(columns = {1:'iid'}, inplace=True)
pop_info.loc[:,'iid']=pop_info.loc[:,'iid'].astype(str)
eigenvecs = pd.merge(eigenvecs, pop_info)

In [ ]:
pops0 = ['ansbach','fleckvieh','namibia','orig_fleckvieh','braunvieh','holstein']
pops1 = ['ansbach','fleckvieh','namibia','orig_fleckvieh']

In [ ]:
colors = ['red','brown','magenta','orange','black','blue']
i = 0
df0 = eigenvecs[eigenvecs['valid']==1]
df0 = eigenvecs
for pop in pops0:
    df = df0[df0['pop']==pop]
    plt.scatter(df[2],df[3],color=colors[i], s=5,label=pop)
    i = i + 1
plt.title('All populations')
plt.xlabel('PC1')
plt.ylabel('PC2')
out=plt.legend(loc=0)

In [ ]:
colors = ['red','brown','magenta','orange','black','blue']
i = 0
df0 = eigenvecs[eigenvecs['valid']==1]
df0 = eigenvecs
for pop in pops1:
    df = df0[df0['pop']==pop]
    plt.scatter(df[2],df[3],color=colors[i], s=15,label=pop)
    i = i + 1
plt.title('Simmental populations')
plt.xlabel('PC1')
plt.ylabel('PC2')
out=plt.legend(loc=0)

In [ ]:
colors = ['black','brown','magenta','orange','black','blue']
i = 0
df0 = eigenvecs[eigenvecs['valid']==1]
#df0 = eigenvecs
for pop in pops1:
    df = df0[df0['pop']==pop]
    plt.scatter(df[2],df[3],color=colors[i], s=15,label=pop)
    i = i + 1
plt.title('Simmental populations pruned')
plt.xlabel('PC1')
plt.ylabel('PC2')
out=plt.legend(loc=0)

Erstellen eine PLINK Dateiensets für die Namibia Simmental Gruppe:

In [ ]:
pop_info = pd.read_excel("{0}/fams/pop_info.xlsx".format(gts_dir))

In [ ]:
namibia_simmental = pop_info[(pop_info['pop']=='namibia') & (pop_info['valid']==1)]

In [ ]:
namibia_simmental[['fid','iid']].to_csv("{0}/namibia_simmental.csv".format(gts_dir), \
                                        sep='\t', header=None, index=False)

In [ ]:
cmd = "plink --cow --bfile {0}/namibia_ARS12 \
--keep {0}/namibia_simmental.csv \
--make-bed --out {0}/namibia_simmental_ARS12".format(gts_dir)

In [ ]:
os.system(cmd)

---

### Identifizierung von SNPs die das Namibia Fleckvieh vom früheren Fleckvieh unterscheiden

In [ ]:
pop_info = pd.read_excel("{0}/fams/pop_info.xlsx".format(gts_dir))

In [ ]:
origFV_namibiaFV = pop_info[((pop_info['pop1']=='orig_fleckvieh') | \
                             (pop_info['pop1']=='namibia'))  & (pop_info['valid']==1) ]

In [ ]:
origFV_namibiaFV[['fid','iid']].to_csv("{0}/fams/origFV_namibiaFV.csv".\
                        format(gts_dir), header=None, index=False,sep='\t')

In [ ]:
cmd = "plink --cow --bfile {0}/five_pops --keep {0}/fams/origFV_namibiaFV.csv \
--geno 0.1 --make-bed --out {0}/origFV_namibiaFV".format(gts_dir)

In [ ]:
os.system(cmd)

Umwandeln von PLINK Datei in eine Text-Datei mit den Genotypbezeichnungen 0,1 und 2, codierend nur für die Autosomen:

In [ ]:
cmd = "plink --cow --bfile {0}/origFV_namibiaFV --chr 1-29 --recode A \
--out {0}/origFV_namibiaFV".format(gts_dir)

In [ ]:
os.system(cmd)

In [ ]:
orig_namibia_df = pd.read_csv("{0}/origFV_namibiaFV.raw".format(gts_dir), \
                              delim_whitespace=True, low_memory=False)

In [ ]:
snps = pd.Series(orig_namibia_df.columns[6:])

In [ ]:
#Genotype array
X = np.array(orig_namibia_df.iloc[:,6:])

In [ ]:
#Fill nan with allele frequency
#Use np.mean does not work - use np.nanmean
#Allele frequency is mean of genotype values of each column divided by 2
freqs = np.nanmean(X, axis=0)/2
na_pos = np.where(np.isnan(X))
X[na_pos] = np.take(freqs, na_pos[1]) 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
pca.fit(X)

In [ ]:
Y = pca.transform(X)

In [ ]:
plt.bar(range(1,11), pca.explained_variance_ratio_[:10])
plt.title('PCA variance explained')
plt.xlabel('PCA no.')
out=plt.ylabel('Proportion of variance explained')

Hinzufügen der Informationen der Rassen zu den Genotyp- und PCA-Datenrahmen:

In [ ]:
pop_info = pd.read_excel("{0}/fams/pop_info.xlsx".format(gts_dir))
pop_info.rename(columns={'iid':'IID'}, inplace=True)
orig_namibia_df.loc[:,'IID'] = orig_namibia_df.loc[:,'IID'].astype(str)
pop_info.loc[:,'IID'] = pop_info.loc[:,'IID'].astype(str)

In [ ]:
merged = pd.merge(orig_namibia_df, pop_info)[['IID','pop1']]

In [ ]:
Y_df = pd.DataFrame(Y)

In [ ]:
Y_df1 = pd.concat([merged,Y_df], axis=1)

In [ ]:
orig_fv = Y_df1[Y_df1.pop1=='orig_fleckvieh']
namibia = Y_df1[Y_df1.pop1=='namibia']

In [ ]:
plt.scatter(namibia[0], namibia[1], color='black', label='namibia')
plt.scatter(orig_fv[0], orig_fv[1], color='orange', label='orig_fv')
out=plt.legend()

Die Bedeutung einzelner SNPs bei der Unterscheidung der beiden Rassen:

In [ ]:
loading_scores=pd.Series(pca.components_[0])
loading_scores_sorted = loading_scores.abs().sort_values(ascending=False)

Kombinieren der `loading_scores` mit den SNP IDs:

In [ ]:
SNP = pd.concat([loading_scores_sorted, snps], \
                axis=1).sort_values([0], ascending=False)

In [ ]:
plt.plot(range(0,1000), SNP[0][:1000])
plt.title('SNP loadings (1st PC)')
plt.xlabel('SNP')
out=plt.ylabel('Loading')

Zusammenfassend trägt nicht nur ein einzelner SNP zur Unterscheidung der beiden Rassen bei, sondern ca. 200 SNPs.

---

### Identifizierung von Selektionssignaturen

Zu Beginn werden die nötigen Pakete, wie `os`, `pandas` und `numpy` importiert:

In [ ]:
import os
import pandas as pd
import numpy as np

`HOME` wird als die lokale Umgebung definiert, damit ein Zugriff darauf möglich ist. `gts_dir` ist fortan die Variable mit allen Genotypinformationen.

In [ ]:
HOME = os.getenv('HOME')
gts_dir = "{}/GTS/".format(HOME)

Im Nachfolgenden werden die Genotypinformationen aller Populationen zusammengefasst:

In [ ]:
pop_info = pd.read_excel("{0}/fams/pop_info.xlsx".format(gts_dir))
pop_info.loc[:,'iid'] = pop_info.loc[:,'iid'].astype(str)
five_pops = pd.read_csv("{0}/five_pops.fam".format(gts_dir), delim_whitespace=True, header=None)
five_pops.rename(columns={1:'iid'}, inplace=True)

Hierbei ist darauf zu achten, dass die zweite Spalte in der Datei `five_pops` mit `iid` (individual ID) betitelt wird. Dies wird bei der weiteren Bearbeitung der Daten mit Hilfe des Pakets `PLINK` (v1.90b6.10) [(Purcell et al. 2007)](https://doi.org/10.1086/519795) von Bedeutung.  
<br>
`pop_info` wird als `.xlsx` und `five_pops` als `.fam` Datei gespeichert.

Nachfolgend werden die individuellen Daten der verschiedenen Populationen aus der Datei `pop_info.xlsx` extrahiert und für jede Population z.B. als `namibiaFV.cluster` gespeichert:

In [ ]:
pop_info[(pop_info.valid==1)&(pop_info.pop1=='namibia')]\
                            [['iid']].to_csv("{0}/namibiaFV.cluster".\
                            format(gts_dir), sep='\t', index=False, header=None)
pop_info[(pop_info.valid==1)&(pop_info.pop1=='orig_fleckvieh')]\
                            [['iid']].to_csv("{0}/origFV.cluster".\
                            format(gts_dir), sep='\t', index=False, header=None)
pop_info[(pop_info.valid==1)&(pop_info.pop1=='fleckvieh')]\
                            [['iid']].to_csv("{0}/FV.cluster".\
                            format(gts_dir), sep='\t', index=False, header=None)
pop_info[(pop_info.valid==1)&(pop_info.pop1=='holstein')]\
                            [['iid']].to_csv("{0}/HF.cluster".\
                            format(gts_dir), sep='\t', index=False, header=None)
pop_info[(pop_info.valid==1)&(pop_info.pop1=='ansbach')]\
                            [['iid']].to_csv("{0}/ansbach.cluster".\
                            format(gts_dir), sep='\t', index=False, header=None)

Im nächsten Schritt wird ein 'VCF'-Dateienset unter Verwendung der `PLINK` Anwendung erstellt. 'VCF'-Dateien beinhalten komplexe genetische Variationsdaten, die mit Hilfe der Anwendung `VCFtools` (Version 0.1.17) [(Danecek et al. 2011)](https://doi.org/10.1093/bioinformatics/btr330) leicht interpretiert und analysiert werden können.  
<br>
Desweiteren wird eine neue Variable `ref_dir` definiert, in der sich die Referenzdaten befinden:

In [ ]:
ref_dir = "{}/Forschungsprojekt_Namibia_Fleckvieh/REF/ARRAYS/".format(HOME)

In [ ]:
cmd  = "plink \
--cow --bfile {0}/five_pops \
--a2-allele {1}/SNPId_REF_A 2 1 \
--geno 0.1 \
--recode vcf-iid \
--out {0}/five_pops".format(gts_dir, ref_dir)

os.system(cmd)

Es wird exemplarisch an aufgeführtem Befehl die Struktur eines `PLINK` Befehls erklärt:

*  `plink` : eröffnet den Befehl der Anwendung `PLINK`
*  `--cow` : etabliert den Chromosomensatz für Rinder
*  `--bfile` : spezifiziert das Dateienset `.bed`, `.bim` und `.fam`
*  `--a2-allele` : spezifiziert das A2 Allel als Referenzallel
*  `--geno` : hier wird mit der Zahl (0.1) das Maximium an fehlenden Genotypisierung pro SNP festgelegt
*  `--recode vcf-iid` : rekreirt eine 'VCF'-Datei ohne IID
*  `--out` : spezifiziert den Output Dateinamen  
<br>

[(Purcell et al. 2007)](https://doi.org/10.1086/519795)

Mit Hilfe der Anwendung `VCFtools` mit dem Befehl `--weir-fst-pop <filename>` erhält man die $F_{ST}$ Statistik für die jeweiligen Populationen. Mit den Befehlen `--fst-window-size` und `--fst-window-step` werden die Größe des Fensters und die Länge der Schrittgröße festgelegt.  
<br>
<br>
<p style="text-decoration:underline">Im ersten Fall werden die Populationen Original-Fleckvieh und Namibia-Fleckvieh miteinander betrachtet:</p>

In [ ]:
cmd = "vcftools \
--vcf {0}/five_pops.vcf \
--weir-fst-pop {0}/origFV.cluster \
--weir-fst-pop {0}/namibiaFV.cluster \
--fst-window-size 2000000 \
--fst-window-step 500000 \
--out {0}/origFV_vs_namibiaFV".format(gts_dir)

os.system(cmd)

<p style="text-decoration:underline">Im nächsten Fall sind es Fleckvieh und Namibia-Fleckvieh:</p>

In [ ]:
cmd = "vcftools \
--vcf {0}/five_pops.vcf \
--weir-fst-pop {0}/FV.cluster \
--weir-fst-pop {0}/namibiaFV.cluster \
--fst-window-size 2000000 \
--fst-window-step 500000 \
--out {0}/FV_vs_namibiaFV".format(gts_dir)

os.system(cmd)

<p style="text-decoration:underline">Fleckvieh und Original-Fleckvieh:</p>

In [ ]:
cmd = "vcftools \
--vcf {0}/five_pops.vcf \
--weir-fst-pop {0}/FV.cluster \
--weir-fst-pop {0}/origFV.cluster \
--fst-window-size 2000000 \
--fst-window-step 500000 \
--out {0}/FV_vs_origFV".format(gts_dir)

os.system(cmd)

<p style="text-decoration:underline">Fleckvieh und Ansbach-Triesdorfer:</p>

In [ ]:
cmd = "vcftools \
--vcf {0}/five_pops.vcf \
--weir-fst-pop {0}/FV.cluster \
--weir-fst-pop {0}/ansbach.cluster \
--fst-window-size 2000000 \
--fst-window-step 500000 \
--out {0}/FV_vs_ansbach".format(gts_dir)

os.system(cmd)

Jetzt können die Ergebnisse der $F_{ST}$ Statistik ausgelesen werden:

In [ ]:
vcf_fst_origFV_namibia = pd.read_csv("{0}/origFV_vs_namibiaFV.windowed.weir.fst" \
                                     .format(gts_dir),sep='\t')
vcf_fst_FV_namibia     = pd.read_csv("{0}/FV_vs_namibiaFV.windowed.weir.fst" \
                                     .format(gts_dir),sep='\t')
vcf_fst_FV_origFV    = pd.read_csv("{0}/FV_vs_origFV.windowed.weir.fst" \
                                   .format(gts_dir),sep='\t')
vcf_fst_FV_ansbach    = pd.read_csv("{0}/FV_vs_ansbach.windowed.weir.fst" \
                                    .format(gts_dir),sep='\t')

Nun ist es möglich die Mittelwerte, Standardabweichungen und die gewichteten Werte für $F_{ST}$ zu entnehmen, die nach [Cockerham und Weir (1984)](https://doi.org/10.2307/2530754) mit Hilfe der Anwendung `VCFtools` berechnet wurden. Die gewichteten Werte für $F_{ST}$ berücksichtigen die Anzahl an SNPs innerhalb eines Fensters [(Wright 1949](https://doi.org/10.1111/j.1469-1809.1949.tb02451.x), [1980)](https://doi.org/10.1016/0047-2484(80)90079-2).

In [ ]:
m_fst_origFV_namibia = vcf_fst_origFV_namibia.WEIGHTED_FST.mean()
sd_fst_origFV_namibia = vcf_fst_origFV_namibia.WEIGHTED_FST.std()

m_fst_FV_namibia = vcf_fst_FV_namibia.WEIGHTED_FST.mean()
sd_fst_FV_namibia = vcf_fst_FV_namibia.WEIGHTED_FST.std()

m_fst_FV_origFV = vcf_fst_FV_origFV.WEIGHTED_FST.mean()
sd_fst_FV_origFV = vcf_fst_FV_origFV.WEIGHTED_FST.std()

m_fst_FV_ansbach = vcf_fst_FV_ansbach.WEIGHTED_FST.mean()
sd_fst_FV_ansbach = vcf_fst_FV_ansbach.WEIGHTED_FST.std()

In den nächsten Schritten werden die $Z$-Werte der $F_{ST}$ Statistik berechnet und dann jeweils als `.csv` Dateien in `gts_dir` gespeichert:

In [ ]:
vcf_fst_origFV_namibia['Z_FST']= (vcf_fst_origFV_namibia.WEIGHTED_FST-m_fst_origFV_namibia)/sd_fst_origFV_namibia
vcf_fst_FV_namibia['Z_FST']= (vcf_fst_FV_namibia.WEIGHTED_FST-m_fst_FV_namibia)/sd_fst_FV_namibia
vcf_fst_FV_origFV['Z_FST']= (vcf_fst_FV_origFV.WEIGHTED_FST-m_fst_FV_origFV)/sd_fst_FV_origFV
vcf_fst_FV_ansbach['Z_FST']= (vcf_fst_FV_ansbach.WEIGHTED_FST-m_fst_FV_ansbach)/sd_fst_FV_ansbach

In [ ]:
vcf_fst_origFV_namibia.to_csv("{0}Z_FST_origFV_namibia.csv".format(gts_dir), sep='\t',index=False)
vcf_fst_FV_namibia.to_csv("{0}Z_FST_FV_namibia.csv".format(gts_dir), sep='\t',index=False)
vcf_fst_FV_origFV.to_csv("{0}Z_FST_FV_origFV.csv".format(gts_dir), sep='\t',index=False)
vcf_fst_FV_ansbach.to_csv("{0}Z_FST_FV_ansbach.csv".format(gts_dir), sep='\t',index=False)

Unter Verwendung der Anwendung `matplotlib` (Version 3.1.1) [(Hunter 2007)](https://doi.org/10.1109/MCSE.2007.55) ist es möglich die Daten graphisch darzustellen:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.subplots(figsize=(10,10))

plt.subplot(221)
plt.hist(vcf_fst_origFV_namibia.Z_FST, bins=50)
plt.xlabel('$Z$-Werte')
plt.title('Original-Fleckvieh - Namibia-Fleckvieh')

plt.subplot(222)
plt.hist(vcf_fst_FV_namibia.Z_FST, bins=50)
plt.xlabel('$Z$-Werte')
plt.title('Fleckvieh - Namibia-Fleckvieh')

plt.subplot(223)
plt.hist(vcf_fst_FV_origFV.Z_FST, bins=50)
plt.xlabel('$Z$-Werte')
plt.title('Fleckvieh - Original-Fleckvieh')

plt.subplot(224)
plt.hist(vcf_fst_FV_ansbach.Z_FST, bins=50)
plt.xlabel('$Z$-Werte')
plt.title('Fleckvieh - Ansbach-Triesdorfer')


plt.tight_layout()
plt.suptitle('$Z$-Werte der berechneten $F_{ST}$ Statistik', y=1.03, fontsize=20)

Mit Hilfe des `Ensembl Project` [(Cunningham et al. 2019)](https://doi.org/10.1093/nar/gky1113) und des darin befindlichen `Biomart` ist es möglich eine aktuelle Genliste für die Tierart Rind zu generieren.  <br>
Der `Biomart` ermöglicht eine bedarfsgenaue Einstellung der gewünschten Kriterien und eine dementsprechende Ausgabe der Daten.  

Im Folgenden werden die Daten eingelesen und die Spaltenbezeichnungen angepasst:

In [ ]:
genes_expanded = pd.read_csv("{0}/ensembl_bta_genes.txt".format(gts_dir), sep='\t')

In [ ]:
rename_cols = {\
          'Gene stable ID': 'Ensembl_ID' ,\
          'Gene start (bp)': 'start', \
          'Gene end (bp)': 'end', \
          'Gene name': 'Symbol', \
          'Gene Synonym': 'Synonym', \
          'Chromosome/scaffold name': 'Chromosome', \
          'GOSlim GOA Description': 'GO'}

genes_expanded = genes_expanded.rename(columns=rename_cols)

Es wird die Ausgabe in so fern angepasst, dass jedes Gen nur einmalig aufgeführt ist und alle Beschreibungen aus der Spalte 'GOSlim GOA Description' aufgeführt werden:

In [ ]:
listGO = lambda x: "; ".join(x.iloc[:])
getUnique = lambda x: x.iloc[0]
genes=genes_expanded.groupby(['Ensembl_ID']).agg({ \
    'Symbol': getUnique, \
    'start': getUnique, \
    'end': getUnique, \
    'Synonym': getUnique, \
    'Chromosome': getUnique, \
    'GO': listGO}).reset_index()

Zur leichteren Interpretation in der Genetik wird häufig ein sogenannter Manhattan-Plot verwendet, in dem jedes Chromosom und die dazugehörigen Signalstärken der SNPs verdeutlicht werden.  
<br>
Deshalb wird im Folgenden eine Anwendung des Manhattan-Plots auf die bereits berechneten $Z$-Werte der $F_{ST}$ Werte erstellt:

In [ ]:
%%writefile manhattan_fst.py
def manhattan(fst_file):
    
    import matplotlib.pyplot as plt
    import numpy as np
    from itertools import cycle
    import pandas as pd

    ps = pd.read_csv(fst_file, delim_whitespace=True)
    #ps['BP']= ps['SNP'].apply(lambda x : x.split('_')[1])
    ps = ps.rename(columns={'Z_FST':'P','CHROM':'CHR', 'BIN_START':'BP'})

    ps = ps.sort_values(['CHR', 'BP'])
    ps = ps[ps.CHR>0]
    chrs = list(set(list(ps['CHR'])))

    #print(len(chrs))

    BP_base = 0
    colors = cycle(['black', 'grey'])
    chr_ticks = []
    bonf = 4

    plt.figure(figsize=(30,8))

    for chr in chrs[:-1]:
        col = next(colors)
        BP0 = ps[ps['CHR'] == chr].BP
        BP  = ps[ps['CHR'] == chr].BP.astype(int)  + BP_base
        P = abs(ps[ps['CHR']==chr].P)
        chr_tick = ((max(BP) - BP_base)/2) + BP_base
        chr_ticks.append(chr_tick)
        plt.scatter(BP[P<bonf],P[P<bonf], s=50, alpha=1, color=col, edgecolors='none')
        plt.scatter(BP[P>=bonf],P[P>=bonf], s=50, alpha=1, color='red', edgecolors='none')
        BP_base = max(BP)

    if len(chrs)>30:
        CHRS = ["{0}".format(x) for x in list(chrs)][:-2]
        CHRS.append('X')
    else:
        CHRS = ["{0}".format(x) for x in list(chrs)][:-1]
    

    plt.xticks(chr_ticks, CHRS, rotation=0)
    ax = plt.gca()
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.tick_params(labelsize =15)
    plt.xlabel('Chromosome', fontsize=15)
    plt.ylabel('Z_FST', fontsize=15)
    
    plt.hlines(bonf, 0, BP_base, linestyle='dashed',color='grey')

    plt.xlim((0, BP_base))
    plt.ylim((0,10))

    plt.show(block='False')

Nun ist es möglich die Anwendung `manhattan_fst` zu importieren und auf unsere Rassen anzuwenden:

In [ ]:
import manhattan_fst
import importlib
importlib.reload(manhattan_fst)

<p style="text-decoration:underline">Zunächst der Vergleich zwischen Original-Fleckvieh und Namibia-Fleckvieh:</p>

In [ ]:
manhattan_fst.manhattan("{0}Z_FST_origFV_namibia.csv".format(gts_dir))

<p style="text-decoration:underline">Fleckvieh und Namibia-Fleckvieh:</p>

In [ ]:
manhattan_fst.manhattan("{0}Z_FST_FV_namibia.csv".format(gts_dir))

<p style="text-decoration:underline">Fleckvieh und Original-Fleckvieh:</p>

In [ ]:
manhattan_fst.manhattan("{0}Z_FST_FV_origFV.csv".format(gts_dir))

<p style="text-decoration:underline">Fleckvieh und Ansbach-Triesdorfer:</p>

In [ ]:
manhattan_fst.manhattan("{0}Z_FST_FV_ansbach.csv".format(gts_dir))

Um sich die positionellen Kandidatengene anzeigen zu lassen wird folgende Funktion definiert:

In [ ]:
def get_candidates(df, keyword):
    top = df[df.Z_FST>=4.0]
    top = top.reset_index()
    dfs = []
    for i in top.index:
        chrom = top.iloc[i].CHROM
        if chrom != 0:
            if chrom == 30:
                chrom = "X"
            else:
                chrom = str(int(chrom))
            start = top.iloc[i].BIN_START
            #print(chrom,start)
            out = (genes[(genes.Symbol.notnull()) \
              & (genes.Chromosome==chrom) \
              & (genes.start > start) \
              & (genes.end < start + 2000000) \
              & (genes.GO.str.contains(keyword))])
            if not out.empty:
                dfs.append(out)
    result = pd.concat(dfs)
    result = result.drop_duplicates(['Symbol'])
    return result

Mit Hilfe dieser Funktion lassen sich Kandidatengene mit einem bestimmten Schlüsselwort kombinieren wodurch folglich die Gene, die in Verbindung mit der gewünschten Eigenschaft stehen angezeigt werden. In den nachfolgenden Schritten handelt es sich dabei um die Eigenschaft 'Pigmentation'.

<p style="text-decoration:underline">Zunächst wieder für den Vergleich Original-Fleckvieh und Namibia-Fleckvieh:</p>

In [ ]:
get_candidates(vcf_fst_origFV_namibia, 'pigmentation')

<p style="text-decoration:underline">Fleckvieh und Namibia-Fleckvieh:</p>

In [ ]:
get_candidates(vcf_fst_FV_namibia, 'pigmentation')

<p style="text-decoration:underline">Fleckvieh und Original-Fleckvieh:</p>

In [ ]:
get_candidates(vcf_fst_FV_origFV, 'pigmentation')

<p style="text-decoration:underline">Fleckvieh und Ansbach-Triesdorfer:</p>

In [ ]:
get_candidates(vcf_fst_FV_ansbach, 'pigmentation')

---

### Berecnnung zum Erhalt der Rasse Frequenzen

Vorbereiten der Daten `GPR143_run7.csv`, `GPR143.csv`, `MITF_run7.csv` und `MITF.csv`

In [ ]:
import pandas as pd

In [ ]:
gpr143_run7 = pd.read_csv('GPR143_run7.csv', sep='\t', header=None)
gpr143 = pd.read_csv('GPR143.csv', sep='\t', header=None)
mitf_run7 = pd.read_csv('MITF_run7.csv', sep='\t', header=None)
mitf = pd.read_csv('MITF.csv', sep='\t', header=None)

Erstellen eines dictionary zur Veränderung der Spaltenbezeichnungen:

In [ ]:
rename_cols = \
{0:'Chromosome',\
 1:'Position',\
 2:'Symbol',\
 3:'Region',\
 4:'AA_1',\
 5:'AA_2',\
 6:'Effect',\
 7:'Frequency',\
 8:'RS-Nr.',\
 9:'Source',\
10:'SIFT'}

Zusammenfassen der Daten der TUM und der Daten aus dem RUN7 des 1000 Bullen Projekts:

In [ ]:
dataRUN7 = (gpr143_run7, mitf_run7)
dataTUM = (gpr143, mitf)

Anwenden des dictionary auf alle `.csv` Dateien und hinzufügen einer Spalte 'Type' mit der jeweilgen Bezeichnung wo die Daten erhoben wurden:

In [ ]:
for i in dataRUN7:
    i.rename(columns=rename_cols, inplace=True)
    i['Type']='RUN7'
    
for j in dataTUM:
    j.rename(columns=rename_cols, inplace=True)
    j['Type']='TUM'

Zusammenfassen aller vier `.csv` Dateien in einer Datei `allruns.csv`:

In [ ]:
pd.concat([gpr143_run7, gpr143, mitf_run7, mitf], ignore_index=True).to_csv('allruns.csv', sep='\t')

Eigentliche Berechnung der Frequenzen

In [ ]:
allruns = pd.read_csv('allruns.csv', sep='\t')
allruns.loc[:, 'Frequency'] = allruns.loc[:, 'Frequency'].astype(float)
allruns.rename(columns={'Region':'Mut_type', 'Type':'Project'}, inplace=True)
allruns = allruns.drop(allruns.columns[[0]], axis=1)

In [ ]:
MITF_RUN7 = allruns[(allruns.Project=='RUN7') \
                      & (allruns.Symbol=='MITF')
                      & ~(allruns.Effect.str.contains('MODI')) \
                      & (allruns.Mut_type != 'synonymous') \
                      & (allruns.Frequency >= 0.001)]
mitf_run7_variants = ["{0}_{1}".format(x[0],x[1]) for x in zip(MITF_RUN7.Chromosome, MITF_RUN7.Position)]

GPR143_RUN7 = allruns[(allruns.Project=='RUN7') \
                      & (allruns.Symbol=='GPR143')
                      & ~(allruns.Effect.str.contains('MODI')) \
                      & (allruns.Mut_type != 'synonymous') \
                      & (allruns.Frequency >= 0.001)]
gpr143_run7_variants = ["{0}_{1}".format(x[0],x[1]) for x in zip(GPR143_RUN7.Chromosome, GPR143_RUN7.Position)]

MITF_TUM = allruns[(allruns.Project=='TUM') \
                      & (allruns.Symbol=='MITF')
                      & ~(allruns.Effect.str.contains('MODI')) \
                      & (allruns.Mut_type != 'synonymous') \
                      & (allruns.Frequency >= 0.001)]
mitf_tum_variants = ["{0}_{1}".format(x[0],x[1]) for x in zip(MITF_TUM.Chromosome, MITF_TUM.Position)]

GPR143_TUM = allruns[(allruns.Project=='TUM') \
                      & (allruns.Symbol=='GPR143')
                      & ~(allruns.Effect.str.contains('MODI')) \
                      & (allruns.Mut_type != 'synonymous') \
                      & (allruns.Frequency >= 0.001)]
gpr143_tum_variants = ["{0}_{1}".format(x[0],x[1]) for x in zip(GPR143_TUM.Chromosome, GPR143_TUM.Position)]

In [ ]:
import os
import pandas as pd
dfs = []
for variant in mitf_run7_variants:
    chrom_pos = (variant.split('_')[0], variant.split('_')[1])
    cur_dir = os.getcwd()
    cmd = "cd /home/{0}/YSERVE/veps_run7/ && exec ./l_get_gts.py {1} {2} > \
    {3}/freqs.csv".format(local_user,chrom_pos[0], chrom_pos[1], cur_dir)
    os.system(cmd)
    freqs = pd.read_csv("./freqs.csv", sep='|',header=None, skiprows=5, skipfooter=1, \
                        engine='python')
    breed_freq = freqs.loc[:,[1,freqs.shape[1]-3]]
    freq_column = breed_freq.columns[-1]
    breed_freq.rename(columns={freq_column:'freq'}, inplace=True)
    breed_freq['variant']="{0}_{1}".format(chrom_pos[0],chrom_pos[1])
    dfs.append(breed_freq)
df = pd.concat(dfs)
df.loc[:,1] = df[1].str.strip()
df.rename(columns = {1:'breed'}, inplace=True)
breed_freq = (df.pivot(index='breed', columns='variant'))
variants = [x[1] for x in breed_freq.columns]
breeds = breed_freq.index.tolist()

Mit `seaborn` ist es möglich die Daten gut interpretierbar darzustellen.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Kurze Erklärung zu nachfolgendem Befehl:  
Die Argumente von `sns.heatmap()` sind gut verständlich, doch einige habe ich nicht gleich verstanden:
+ `linewidth` und `linecolor` geben ein Grid an (macht es meiner Meinung nach übersichtlicher)
+ `annot=True` macht die Frequenzen, also die direkten Zahlenwerte sichtbar
+ `fmt=.3f` gibt an wie viele Stellen der Frequenzen angegeben werden; hier: 3 Nachkommastellen als Datentyp float
+ `cmap=YlOrRd` gibt die Farbpalette an
+ `mask=breed_freq==0.000` maskiert alle Werte gleich 0.000, kann auch mit < oder > gemacht werden

Die obere und untere Reihe waren nur halb sichtbar, was mit folgenden Befehlen gelöst wurde:
+ `bottom, top = ax.get_ylim()`
+ `ax.set_ylim(bottom + 0.5, top - 0.5)`

In [ ]:
plt.figure(figsize=(15,25))
plt.title('Frequencies of MITF_RUN7 variants', size=15)
ax = sns.heatmap(breed_freq, linewidths=0.1, linecolor='black', annot=True, fmt='.3f',\
                 cmap='YlOrRd', mask=breed_freq==0.000)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
ax.set(xlabel='Variants', ylabel='Breed')
plt.savefig('Freq_heatmap_MITF_RUN7.png')

In [ ]:
import os
import pandas as pd
dfs = []
for variant in gpr143_run7_variants:
    chrom_pos = (variant.split('_')[0], variant.split('_')[1])
    cur_dir = os.getcwd()
    cmd = "cd /home/{0}/YSERVE/veps_run7/ && exec ./l_get_gts.py {1} {2} > \
    {3}/freqs.csv".format(local_user,chrom_pos[0], chrom_pos[1], cur_dir)
    os.system(cmd)
    freqs = pd.read_csv("./freqs.csv", sep='|',header=None, skiprows=5, skipfooter=1, \
                        engine='python')
    breed_freq = freqs.loc[:,[1,freqs.shape[1]-3]]
    freq_column = breed_freq.columns[-1]
    breed_freq.rename(columns={freq_column:'freq'}, inplace=True)
    breed_freq['variant']="{0}_{1}".format(chrom_pos[0],chrom_pos[1])
    dfs.append(breed_freq)
df = pd.concat(dfs)
df.loc[:,1] = df[1].str.strip()
df.rename(columns = {1:'breed'}, inplace=True)
breed_freq = (df.pivot(index='breed', columns='variant'))
variants = [x[1] for x in breed_freq.columns]
breeds = breed_freq.index.tolist()

In [ ]:
plt.figure(figsize=(25,30))
plt.title('Frequencies of GPR143_RUN7 variants', size=15)
ax = sns.heatmap(breed_freq, linewidths=0.1, linecolor='black', annot=True, fmt='.3f', \
                 cmap='YlOrRd', mask=breed_freq==0.000)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
ax.set(xlabel='Variants', ylabel='Breed')
plt.savefig('Freq_heatmap_GPR143_RUN7.png')

In [ ]:
import os
import pandas as pd
dfs = []
for variant in mitf_tum_variants:
    chrom_pos = (variant.split('_')[0], variant.split('_')[1])
    cur_dir = os.getcwd()
    cmd = "cd /home/{0}/YSERVE/veps/ && exec ./l_get_gts.py {1} {2} > \
    {3}/freqs.csv".format(local_user,chrom_pos[0], chrom_pos[1], cur_dir)
    os.system(cmd)
    freqs = pd.read_csv("./freqs.csv", sep='|',header=None, skiprows=5, skipfooter=1, \
                        engine='python')
    breed_freq = freqs.loc[:,[1,freqs.shape[1]-3]]
    freq_column = breed_freq.columns[-1]
    breed_freq.rename(columns={freq_column:'freq'}, inplace=True)
    breed_freq['variant']="{0}_{1}".format(chrom_pos[0],chrom_pos[1])
    dfs.append(breed_freq)
df = pd.concat(dfs)
df.loc[:,1] = df[1].str.strip()
df.rename(columns = {1:'breed'}, inplace=True)
breed_freq = (df.pivot(index='breed', columns='variant'))
variants = [x[1] for x in breed_freq.columns]
breeds = breed_freq.index.tolist()

In [ ]:
plt.figure(figsize=(10,20))
plt.title('Frequencies of MITF_TUM variants', size=15)
ax = sns.heatmap(breed_freq, linewidths=0.1, linecolor='black', annot=True, fmt='.3f', \
                 cmap='YlOrRd', mask=breed_freq==0.000)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
ax.set(xlabel='Variants', ylabel='Breed')
plt.savefig('Freq_heatmap_MITF_TUM.png')

In [ ]:
import os
import pandas as pd
dfs = []
for variant in gpr143_tum_variants:
    chrom_pos = (variant.split('_')[0], variant.split('_')[1])
    cur_dir = os.getcwd()
    cmd = "cd /home/{0}/YSERVE/veps/ && exec ./l_get_gts.py {1} {2} > \
    {3}/freqs.csv".format(local_user,chrom_pos[0], chrom_pos[1], cur_dir)
    os.system(cmd)
    freqs = pd.read_csv("./freqs.csv", sep='|',header=None, skiprows=5, skipfooter=1, \
                        engine='python')
    breed_freq = freqs.loc[:,[1,freqs.shape[1]-3]]
    freq_column = breed_freq.columns[-1]
    breed_freq.rename(columns={freq_column:'freq'}, inplace=True)
    breed_freq['variant']="{0}_{1}".format(chrom_pos[0],chrom_pos[1])
    dfs.append(breed_freq)
df = pd.concat(dfs)
df.loc[:,1] = df[1].str.strip()
df.rename(columns = {1:'breed'}, inplace=True)
breed_freq = (df.pivot(index='breed', columns='variant'))
variants = [x[1] for x in breed_freq.columns]
breeds = breed_freq.index.tolist()

In [ ]:
plt.figure(figsize=(10,15))
plt.title('Frequencies of GPR143_TUM variants', size=15)
ax = sns.heatmap(breed_freq, linewidths=0.1, linecolor='black', annot=True, fmt='.3f', \
                 cmap='YlOrRd', mask=breed_freq==0.000)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
ax.set(xlabel='Variants', ylabel='Breed')
plt.savefig('Freq_heatmap_GPR143_TUM.png')

---

### Berechnung der Allelfrequenzen

Bemerkung: erste eckige Klammer sind alle homozygoten Tiere; die zweite eckige Klammer alle heterozygoten Tiere; die dritte eckige Klammer alle alternativ homzygoten Tiere

In [ ]:
# für alle Tiere
gts = np.array([[57],[1],[0]])

In [ ]:
# für Ansbach Triesdorfer Rind
gts = np.array([[29],[1],[0]])

In [ ]:
# für Namibia Fleckvieh
gts = np.array([[28],[0],[0]])

In [ ]:
p = ((gts[0]*2)+gts[1])/(2*sum(gts))
q = 1-p
print (p,q)

### Berechnung des Hardy-Weinberg Gleichgewichts und der Chi-Quadrat Statistik aus den Allelfrequenzen

In [ ]:
from __future__ import division
import scipy.stats as stats
import numpy as np
import pandas as pd

In [ ]:
def hwe(gts):
    p = ((gts[0]*2)+gts[1])/(2*sum(gts))
    q = 1-p
    print (p,q)
    e_gts = np.array([p**2, 2*p*q,q**2])*sum(gts)
    stat = sum(((gts-e_gts)**2)/(e_gts))
    print (stat)
    return 1-stats.chi2.cdf(stats.chisquare(gts,e_gts)[0],1)

In [ ]:
hwe(gts)

---

### Berechnung des Fisher's Exact Test

In [ ]:
from __future__ import division
import scipy.stats as stats
import numpy as np

Bemerkung: In den eckigen Klammern befinden sich jeweils die Anzahl an homozygoten und heterozygoten Tieren der beiden Subpopulationen

In [ ]:
oddsratio, pvalue = stats.fisher_exact([[32, 2], [27, 7]])
pvalue